# DEVELOPING THE SUDOKU GENERATOR

## Problem Formulation

Importing the libraries:

In [1]:
import numpy as np
from random import choice

Auxiliary functions:

In [2]:
# Defining the constants
N = 9  # number of grid rows and columns
STEP = 3  # sets the size of the quadrant side


# Turns a number between 0 and 80 into a sudoku grid coordinate
def flatten_position(flat_pos):
    return flat_pos // N, flat_pos % N

Checks if the grid satisfies the single element constraint and if the target state has been reached:

In [3]:
def satisfies_constraints(grid_slice):  # the slice is flattened before sorting
    return np.array_equal(np.sort(grid_slice, axis=None), np.arange(1, N+1))

def check_rows(grid):
    for row in grid:
        if not satisfies_constraints(row):
            return False
    
    return True

def check_columns(grid):
    for column in grid.T:
        if not satisfies_constraints(column):
            return False
    
    return True

def check_quadrants(grid):
    for i in range(0, N, STEP):
        for j in range(0, N, STEP):
            if not satisfies_constraints(grid[j : j+STEP, i : i+STEP]):
                return False
    
    return True

def objective_grid(grid):
    return check_rows(grid) and check_columns(grid) and check_quadrants(grid)

Checking available actions for a given grid:

In [4]:
def number_constraint(grid, y, x, num):
    yy = y // STEP * STEP
    xx = x // STEP * STEP
    
    return num not in np.concatenate((grid[y], grid[:, x], grid[yy : yy+STEP, xx : xx+STEP].flatten()))

def available_pos(grid):
    return [(y, x) for y, x in zip(*np.where(grid == 0))]

def available_nums(grid, y, x):
    return [num for num in range(1, N+1) if number_constraint(grid, y, x, num)]

## Solver

Need to implement a solver to check if the grid has a unique solution. To this end, the search strategy with backtracking was used, as follows:

In [5]:
def solver(sudoku_grid):
    grid = sudoku_grid.copy()  # does not change the original grid
    
    solution = None
    
    max_steps = np.count_nonzero(grid==0)
    moves = [None] * max_steps
    actions = [None] * max_steps
    
    size = 0
    while True:
        if objective_grid(grid):
            if not np.all(solution):  # defines the solution, if no one has been found so far
                solution = grid.copy()
            else:  # if there is more than one solution, return None
                return None

        # size == max_steps works like not available_pos(grid) 
        while size == max_steps or not available_nums(grid, *available_pos(grid)[0]):
            size -= 1
            
            while not actions[size]:   
                # There are no more possibilities for actions to be explored
                if size == 0:
                    return solution  # single solution or None

                grid[moves[size]] = 0  # undoing each modification to generate the next successor
                size -= 1

            num = actions[size].pop()

            grid[moves[size]] = num
            size += 1
            
        # Developing subtrees:
        # not at the search limit
        # there are actions available for the current tree level
        pos = available_pos(grid)[0]
        nums = available_nums(grid, *pos)

        num = nums.pop()

        grid[pos] = num
        moves[size] = pos
        actions[size] = nums
        size += 1

Testing the sudoku solver:

In [6]:
# Single solution array
array1 = np.array([[5, 3, 0, 0, 7, 0, 0, 0, 0],
                   [6, 0, 0, 1, 9, 5, 0, 0, 0],
                   [0, 9, 8, 0, 0, 0, 0, 6, 0],
                   [8, 0, 0, 0, 6, 0, 0, 0, 3],
                   [4, 0, 0, 8, 0, 3, 0, 0, 1],
                   [7, 0, 0, 0, 2, 0, 0, 0, 6],
                   [0, 6, 0, 0, 0, 0, 2, 8, 0],
                   [0, 0, 0, 4, 1, 9, 0, 0, 5],
                   [0, 0, 0, 0, 8, 0, 0, 7, 9]])

# Array with more than one solution
array2 = np.array([[5, 3, 0, 0, 7, 0, 0, 0, 0],
                   [6, 0, 0, 1, 9, 5, 0, 0, 0],
                   [0, 9, 8, 0, 0, 0, 0, 6, 0],
                   [8, 0, 0, 0, 6, 0, 0, 0, 3],
                   [4, 0, 0, 8, 0, 3, 0, 0, 1],
                   [7, 0, 0, 0, 2, 0, 0, 0, 6],
                   [0, 6, 0, 0, 0, 0, 2, 8, 0],
                   [0, 0, 0, 4, 1, 9, 0, 0, 5],
                   [0, 0, 0, 0, 8, 0, 0, 0, 0]])

# Unsolved array
array3 = np.array([[5, 3, 0, 0, 7, 0, 1, 0, 0],
                   [6, 0, 0, 1, 9, 5, 0, 0, 0],
                   [0, 9, 8, 0, 0, 0, 0, 6, 0],
                   [8, 0, 0, 0, 6, 0, 0, 0, 3],
                   [4, 0, 0, 8, 0, 3, 0, 0, 1],
                   [7, 0, 0, 0, 2, 0, 0, 0, 6],
                   [0, 6, 0, 0, 0, 0, 2, 8, 0],
                   [0, 0, 0, 4, 1, 9, 0, 0, 5],
                   [0, 0, 0, 0, 8, 0, 0, 7, 9]])

In [7]:
result = solver(array1)

print(result)

[[5 3 4 6 7 8 9 1 2]
 [6 7 2 1 9 5 3 4 8]
 [1 9 8 3 4 2 5 6 7]
 [8 5 9 7 6 1 4 2 3]
 [4 2 6 8 5 3 7 9 1]
 [7 1 3 9 2 4 8 5 6]
 [9 6 1 5 3 7 2 8 4]
 [2 8 7 4 1 9 6 3 5]
 [3 4 5 2 8 6 1 7 9]]


In [8]:
result = solver(array2)

print(result)

None


In [9]:
result = solver(array3)

print(result)

None


## Generator

Fills the grid through the search strategy with backtracking and clears positions checking if the sudoku remains with a unique solution:

In [10]:
def generator(difficulty=41):  # the difficulty parameter represents the number of initial sudoku clues
    grid = np.zeros((N, N), dtype='int8')
    
    actions = [None] * 81
    size = 0  # represents the grid fill level and the current position to be inserted
    while not objective_grid(grid):        
        # Getting the next possible actions
        nums = available_nums(grid, *flatten_position(size))
        
        if nums:
            random_move = np.random.randint(len(nums))
            num = nums.pop(random_move)
            
            actions[size] = nums
        else:
            size -= 1

            while not actions[size]:  # undoing each modification to generate the next successor
                grid[flatten_position(size)] = 0
                size -= 1
            
            random_move = np.random.randint(len(actions[size]))
            num = actions[size].pop(random_move)

        grid[flatten_position(size)] = num
        size += 1
        
    while np.count_nonzero(grid) > difficulty:
        position = choice([(y, x) for y, x in zip(*np.where(grid))])

        grid_copy = grid.copy()
        grid_copy[position] = 0

        if np.all(solver(grid_copy)):  # checks if grid_copy has a unique solution
            grid[position] = 0

    return grid

Testing the sudoku generator:

In [11]:
sudoku = generator()

print(sudoku, '==>', np.count_nonzero(sudoku))
print()
print(solver(sudoku))

[[6 8 0 5 9 0 0 2 3]
 [9 2 0 0 6 0 0 8 4]
 [4 0 0 3 0 0 0 0 1]
 [7 5 0 0 1 0 3 0 0]
 [2 0 0 0 3 5 8 0 6]
 [0 0 6 0 8 0 0 7 5]
 [0 1 7 0 4 2 6 3 9]
 [0 4 0 6 7 1 0 0 0]
 [8 0 0 0 0 3 0 4 0]] ==> 41

[[6 8 1 5 9 4 7 2 3]
 [9 2 3 1 6 7 5 8 4]
 [4 7 5 3 2 8 9 6 1]
 [7 5 8 4 1 6 3 9 2]
 [2 9 4 7 3 5 8 1 6]
 [1 3 6 2 8 9 4 7 5]
 [5 1 7 8 4 2 6 3 9]
 [3 4 9 6 7 1 2 5 8]
 [8 6 2 9 5 3 1 4 7]]


In [12]:
sudoku = generator()

print(sudoku, '==>', np.count_nonzero(sudoku))
print()
print(solver(sudoku))

[[0 7 0 0 0 0 9 0 0]
 [0 0 8 0 1 6 5 2 3]
 [0 3 6 4 9 0 0 0 7]
 [0 0 0 0 0 7 1 8 5]
 [0 5 4 0 0 0 7 9 6]
 [0 6 7 0 8 5 0 4 0]
 [6 1 0 0 0 0 2 0 0]
 [7 8 5 0 0 9 0 0 1]
 [4 0 2 1 0 3 0 7 8]] ==> 41

[[2 7 1 5 3 8 9 6 4]
 [9 4 8 7 1 6 5 2 3]
 [5 3 6 4 9 2 8 1 7]
 [3 2 9 6 4 7 1 8 5]
 [8 5 4 3 2 1 7 9 6]
 [1 6 7 9 8 5 3 4 2]
 [6 1 3 8 7 4 2 5 9]
 [7 8 5 2 6 9 4 3 1]
 [4 9 2 1 5 3 6 7 8]]


In [13]:
sudoku = generator()

print(sudoku, '==>', np.count_nonzero(sudoku))
print()
print(solver(sudoku))

[[0 0 1 3 6 0 9 0 5]
 [7 6 0 5 0 0 4 0 1]
 [5 0 8 4 0 0 0 3 0]
 [0 7 6 8 5 4 0 0 0]
 [8 5 0 0 0 3 2 0 6]
 [3 0 9 0 0 6 0 0 0]
 [9 0 2 6 7 0 1 0 0]
 [1 0 0 0 0 8 5 0 2]
 [0 4 5 0 3 0 7 9 0]] ==> 41

[[4 2 1 3 6 7 9 8 5]
 [7 6 3 5 8 9 4 2 1]
 [5 9 8 4 1 2 6 3 7]
 [2 7 6 8 5 4 3 1 9]
 [8 5 4 1 9 3 2 7 6]
 [3 1 9 7 2 6 8 5 4]
 [9 8 2 6 7 5 1 4 3]
 [1 3 7 9 4 8 5 6 2]
 [6 4 5 2 3 1 7 9 8]]
